In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [2]:
# --- Config ---
LLAMA32_CONFIG = {
    "vocab_size": 128_256,
    "context_length": 4096,
    "emb_dim": 2048,
    "n_heads": 32,
    "n_layers": 16,
    "hidden_dim": 8192,
    "n_kv_groups": 8,
    "rope_base": 500_000.0,
    "dtype": torch.bfloat16,
    "rope_freq": {
        "factor": 32.0,
        "low_freq_factor": 1.0,
        "high_freq_factor": 4.0,
        "original_context_length": 8192,
    }
}


# RMSNorm (Llama uses RMSNorm)

In [3]:
# --- RMSNorm ---
class RMSNorm(nn.Module):
    def __init__(self, dim, eps=1e-6):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))
    def forward(self, x):
        norm = x.pow(2).mean(-1, keepdim=True)
        return x * torch.rsqrt(norm + self.eps) * self.weight


## Rotary Positional Embeddings (RoPE)

In [4]:
# --- RoPE ---
class RotaryEmbedding(nn.Module):
    def __init__(self, dim, base=500_000):
        super().__init__()
        self.dim = dim
        self.base = base
        inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer("inv_freq", inv_freq, persistent=False)
    def forward(self, seq_len, device, dtype):
        t = torch.arange(seq_len, device=device, dtype=dtype)
        freqs = torch.einsum("i,j->ij", t, self.inv_freq.to(dtype))
        return torch.cos(freqs), torch.sin(freqs)

def apply_rope(x, cos, sin):
    cos = cos.to(x.dtype)
    sin = sin.to(x.dtype)
    x1, x2 = x[..., ::2], x[..., 1::2]
    return torch.cat([x1 * cos - x2 * sin, x1 * sin + x2 * cos], dim=-1)


# Llama Feed-Forward Network (SwiGLU)

In [5]:
# --- FeedForward ---
class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.gate = nn.Linear(config["emb_dim"], config["hidden_dim"], bias=False)
        self.up = nn.Linear(config["emb_dim"], config["hidden_dim"], bias=False)
        self.down = nn.Linear(config["hidden_dim"], config["emb_dim"], bias=False)
    def forward(self, x):
        return self.down(F.silu(self.gate(x)) * self.up(x))

# Grouped-Query Multi-Head Attention (GQA)

In [6]:
# --- MultiHeadAttention ---
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.emb_dim = config["emb_dim"]
        self.n_heads = config["n_heads"]
        self.n_kv = config["n_kv_groups"]
        self.head_dim = self.emb_dim // self.n_heads

        self.q_proj = nn.Linear(self.emb_dim, self.emb_dim, bias=False)
        self.k_proj = nn.Linear(self.emb_dim, self.n_kv * self.head_dim, bias=False)
        self.v_proj = nn.Linear(self.emb_dim, self.n_kv * self.head_dim, bias=False)
        self.o_proj = nn.Linear(self.emb_dim, self.emb_dim, bias=False)

        self.rope = RotaryEmbedding(self.head_dim, base=config["rope_base"])

    def forward(self, x):
        B, T, C = x.shape
        dtype = x.dtype

        # Project QKV
        q = self.q_proj(x).view(B, T, self.n_heads, self.head_dim)
        k = self.k_proj(x).view(B, T, self.n_kv, self.head_dim)
        v = self.v_proj(x).view(B, T, self.n_kv, self.head_dim)

        # RoPE
        cos, sin = self.rope(T, x.device, dtype)
        cos, sin = cos[None, :, None, :], sin[None, :, None, :]
        q = apply_rope(q, cos, sin)
        k = apply_rope(k, cos, sin)

        # Expand KV for grouped query
        k = k.repeat_interleave(self.n_heads // self.n_kv, dim=2)
        v = v.repeat_interleave(self.n_heads // self.n_kv, dim=2)

        # Transpose for attention
        q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)

        # Attention scores
        attn_scores = (q @ k.transpose(-2, -1)) / math.sqrt(self.head_dim)

        # Causal mask
        causal_mask = torch.tril(torch.ones(T, T, device=x.device, dtype=torch.bool))
        attn_scores = attn_scores.masked_fill(~causal_mask, float("-inf"))

        # Softmax & output
        attn_probs = F.softmax(attn_scores, dim=-1)
        out = attn_probs @ v
        out = out.transpose(1,2).contiguous().view(B, T, C)
        return self.o_proj(out)

## Transfomer Blocks → Llama 3.2 Model

In [7]:
# --- Transformer Block ---
class TransformerBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.attn_norm = RMSNorm(config["emb_dim"])
        self.ffn_norm = RMSNorm(config["emb_dim"])
        self.attn = MultiHeadAttention(config)
        self.ffn = FeedForward(config)
    def forward(self, x):
        x = x + self.attn(self.attn_norm(x))
        x = x + self.ffn(self.ffn_norm(x))
        return x

# =========================
# 6️⃣ Llama32Model
# =========================
class Llama32Model(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embed = nn.Embedding(config["vocab_size"], config["emb_dim"])
        self.blocks = nn.ModuleList([TransformerBlock(config) for _ in range(config["n_layers"])])
        self.norm = RMSNorm(config["emb_dim"])
        self.lm_head = nn.Linear(config["emb_dim"], config["vocab_size"], bias=False)
    def forward(self, input_ids):
        x = self.embed(input_ids)
        for block in self.blocks:
            x = block(x)
        x = self.norm(x)
        return self.lm_head(x)

# Test dummy input + Load the model on hardware

In [8]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


Using device: cuda


In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = Llama32Model(LLAMA32_CONFIG).to(device).to(torch.bfloat16)
dummy_input = torch.randint(
    0,
    LLAMA32_CONFIG["vocab_size"],
    (2, 128),
    device=device
)

with torch.no_grad():
    logits = model(dummy_input)

print("Output shape:", logits.shape)


Output shape: torch.Size([2, 128, 128256])


# Tokenizer from hugging face

In [10]:
!pip install -q huggingface_hub transformers accelerate

from huggingface_hub import login
login()



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
from transformers import AutoTokenizer

model_name = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Tokenizer loaded!")



Tokenizer loaded!


# encode a sample sentence into tokens and Decode back to text

In [12]:
text = "Hello world! This is a test of the LLaMA 3.2 tokenizer."

encoded = tokenizer(
    text,
    return_tensors="pt"
)

print("Token IDs:", encoded["input_ids"])


decoded = tokenizer.decode(
    encoded["input_ids"][0],
    skip_special_tokens=True
)

print("Decoded text:", decoded)


Token IDs: tensor([[128000,   9906,   1917,      0,   1115,    374,    264,   1296,    315,
            279,    445,   8921,   4940,    220,     18,     13,     17,  47058,
             13]])
Decoded text: Hello world! This is a test of the LLaMA 3.2 tokenizer.


## tokenizer test for edge cases

In [13]:
from transformers import AutoTokenizer

TOKEN = ""
model_name = "meta-llama/Llama-3.2-3B"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=TOKEN
)

# 🔑 Explicitly set PAD = EOS (both token and id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

# ---- Edge case batch test ----
texts = ["", " ", "Hello", "🚀"]

encoded_batch = tokenizer(
    texts,
    return_tensors="pt",
    padding=True,
    truncation=True
)

decoded_batch = [
    tokenizer.decode(ids, skip_special_tokens=True)
    for ids in encoded_batch["input_ids"]
]

print("pad_token:", tokenizer.pad_token)
print("pad_token_id:", tokenizer.pad_token_id)
print("input_ids:\n", encoded_batch["input_ids"])
print("decoded:", decoded_batch)


pad_token: <|end_of_text|>
pad_token_id: 128001
input_ids:
 tensor([[128000, 128001, 128001, 128001],
        [128000,    220, 128001, 128001],
        [128000,   9906, 128001, 128001],
        [128000,   9468,    248,    222]])
decoded: ['', ' ', 'Hello', '🚀']


### Access to the Llama 3.2 weights + simple test model prediction

In [14]:
!pip install transformers accelerate --quiet


In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

HF_TOKEN = ""   # paste new token here
MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    token=HF_TOKEN
)

tokenizer.pad_token = tokenizer.eos_token

# Load model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    token=HF_TOKEN,
    torch_dtype=torch.bfloat16,
    device_map="auto"   # recommended
)

model.eval()
print("Model and tokenizer loaded!")


Device: cuda


`torch_dtype` is deprecated! Use `dtype` instead!


Model and tokenizer loaded!


## Model prediction

In [16]:
input_text = "Once upon a time,"
inputs = tokenizer(input_text, return_tensors="pt").to(device)


In [17]:
with torch.no_grad():
    logits = model(**inputs).logits
    next_token_logits = logits[:, -1, :]  # last token
    next_token_id = torch.argmax(next_token_logits, dim=-1)

predicted_token = tokenizer.decode(next_token_id)
print("Next token prediction:", predicted_token)


Next token prediction:  in


In [18]:
def predict_next_tokens(prompt, max_new_tokens=10):
    """
    Generate tokens iteratively using greedy decoding.

    Args:
        prompt (str): Input text.
        max_new_tokens (int): Number of tokens to predict.

    Returns:
        str: Generated text (prompt + new tokens)
    """
    # Encode prompt
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    for _ in range(max_new_tokens):
        with torch.no_grad():
            logits = model(input_ids).logits  # [batch, seq_len, vocab_size]
            next_token_logits = logits[:, -1, :]  # last token logits
            next_token_id = torch.argmax(next_token_logits, dim=-1)  # greedy
        # Append predicted token
        input_ids = torch.cat([input_ids, next_token_id.unsqueeze(-1)], dim=-1)

    # Decode the entire sequence
    generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return generated_text


In [19]:
prompt = "Once upon a time,"
generated = predict_next_tokens(prompt, max_new_tokens=20)
print("Generated text:\n", generated)


Generated text:
 Once upon a time, in a small village nestled in the rolling hills of the countryside, there lived a young girl named Sophia


## Prepare a small dataset

In [20]:
from torch.utils.data import DataLoader, Dataset

class SmallDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=64):
        self.tokenizer = tokenizer
        self.examples = []
        for t in texts:
            enc = tokenizer(t, truncation=True, max_length=max_length, padding="max_length")
            self.examples.append(torch.tensor(enc["input_ids"]))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]

# Sample dataset
texts = [
    "Once upon a time, there was a brave knight.",
    "The stock market showed significant gains today.",
    "Machine learning can predict trends in healthcare.",
    "Artificial intelligence is transforming science."
]

dataset = SmallDataset(texts, tokenizer)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)


In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.optim import AdamW   # <-- correct
import torch


In [22]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.optim import AdamW

HF_TOKEN = ""
MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"

device = "cuda" if torch.cuda.is_available() else "cpu"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    token=HF_TOKEN
)

tokenizer.pad_token = tokenizer.eos_token

# Model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    token=HF_TOKEN,
    torch_dtype=torch.bfloat16,
    device_map="auto"   # VERY important for 3B
)

model.train()

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)


In [23]:
num_epochs = 20
for epoch in range(num_epochs):
    total_loss = 0
    for batch in dataloader:
        batch = batch.to(device)
        optimizer.zero_grad()

        outputs = model(batch, labels=batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f}")


Epoch 1/20 - Loss: 3.5987
Epoch 2/20 - Loss: 0.2796
Epoch 3/20 - Loss: 0.1406
Epoch 4/20 - Loss: 0.1144
Epoch 5/20 - Loss: 0.0905
Epoch 6/20 - Loss: 0.0785
Epoch 7/20 - Loss: 0.0681
Epoch 8/20 - Loss: 0.0577
Epoch 9/20 - Loss: 0.0475
Epoch 10/20 - Loss: 0.0368
Epoch 11/20 - Loss: 0.0283
Epoch 12/20 - Loss: 0.0242
Epoch 13/20 - Loss: 0.0264
Epoch 14/20 - Loss: 0.0257
Epoch 15/20 - Loss: 0.0261
Epoch 16/20 - Loss: 0.0260
Epoch 17/20 - Loss: 0.0261
Epoch 18/20 - Loss: 0.0254
Epoch 19/20 - Loss: 0.0247
Epoch 20/20 - Loss: 0.0237


## Weights downloaded and loaded into model.

In [24]:
from huggingface_hub import login

HF_TOKEN = ""
login(token=HF_TOKEN)


In [25]:
from huggingface_hub import snapshot_download

repo_id = "meta-llama/Llama-3.2-3B-Instruct"
local_dir = "./llama32_weights"

snapshot_download(repo_id, local_dir=local_dir)


'/content/llama32_weights'

In [26]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer locally
tokenizer = AutoTokenizer.from_pretrained(local_dir)
tokenizer.pad_token = tokenizer.eos_token

# Load model locally
model = AutoModelForCausalLM.from_pretrained(
    local_dir,
    torch_dtype=torch.bfloat16
).to(device)

model.eval()


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,), eps=1e-05)
    (

In [27]:
prompt = "Once upon a time,"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model(**inputs)
    next_token_id = torch.argmax(outputs.logits[:, -1, :], dim=-1)

print("Next token:", tokenizer.decode(next_token_id))


Next token:  in


In [28]:
def predict_next_tokens(prompt, max_new_tokens=10):
    """
    Generate tokens iteratively using greedy decoding.

    Args:
        prompt (str): Input text.
        max_new_tokens (int): Number of tokens to predict.

    Returns:
        str: Generated text (prompt + new tokens)
    """
    # Encode prompt
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    for _ in range(max_new_tokens):
        with torch.no_grad():
            logits = model(input_ids).logits  # [batch, seq_len, vocab_size]
            next_token_logits = logits[:, -1, :]  # last token logits
            next_token_id = torch.argmax(next_token_logits, dim=-1)  # greedy
        # Append predicted token
        input_ids = torch.cat([input_ids, next_token_id.unsqueeze(-1)], dim=-1)

    # Decode the entire sequence
    generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return generated_text


In [29]:
prompt = "Once upon a time,"
generated = predict_next_tokens(prompt, max_new_tokens=20)
print("Generated text:\n", generated)


Generated text:
 Once upon a time, in a small village nestled in the rolling hills of the countryside, there lived a young girl named Sophia


## A simple prompt encode,generate tokens and decode to text

In [30]:
# --- Simple prompt ---
prompt = "Artificial intelligence is transforming"

# --- Encode the prompt ---
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# --- Generate tokens ---
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,      # number of tokens to generate
        do_sample=True,         # enable sampling for creativity
        temperature=0.7,        # control randomness
        pad_token_id=tokenizer.pad_token_id  # ensures padding token is used
    )

# --- Decode generated tokens ---
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Prompt:", prompt)
print("Generated:", generated_text)


Prompt: Artificial intelligence is transforming
Generated: Artificial intelligence is transforming the way we live and work. From smart homes to self-driving cars, AI is being used in various industries to improve efficiency, productivity, and customer experience. However, as AI becomes more prevalent, there are also concerns about its potential impact on jobs


## A clean_text function and produce polished, conversational outputs with chat format.

In [31]:

# --- Chat formatting class ---
class ChatFormat:
    def __init__(self):
        self.messages = []

    def add_message(self, role: str, content: str):
        """Add a message with a role: 'user' or 'assistant'"""
        self.messages.append({"role": role, "content": content})

    def get_prompt(self):
        """Construct the full prompt text for the model"""
        prompt_text = ""
        for msg in self.messages:
            prompt_text += f"{msg['role'].capitalize()}: {msg['content']}\n"
        prompt_text += "Assistant: "
        return prompt_text

# --- Clean text function ---
def clean_text(text: str) -> str:
    """
    Remove repeated EOS tokens, extra whitespaces, and make output readable
    """
    # Remove tokenizer special tokens
    text = text.replace(tokenizer.eos_token, "")
    # Collapse multiple spaces/newlines
    text = " ".join(text.split())
    return text.strip()

# --- Example conversation ---
chat = ChatFormat()
chat.add_message("user", "Hello! Can you explain AI in simple terms?")
chat.add_message("user", "Also, give a quick example in healthcare.")

# --- Build prompt ---
prompt = chat.get_prompt()

# --- Tokenize and generate ---
inputs = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.pad_token_id
    )

# --- Decode and clean ---
raw_output = tokenizer.decode(outputs[0], skip_special_tokens=False)
assistant_reply = raw_output.split("Assistant:")[-1]
assistant_reply = clean_text(assistant_reply)

# --- Show final polished response ---
print("User Prompt:\n", prompt)
print("\nAssistant Reply:\n", assistant_reply)





User Prompt:
 User: Hello! Can you explain AI in simple terms?
User: Also, give a quick example in healthcare.
Assistant: 

Assistant Reply:
 **What is AI?** Artificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as: * Learning * Problem-solving * Reasoning * Perception **How does AI work?** AI systems use algorithms and data to make decisions, predictions, or take actions. These systems can be trained on vast amounts of data, allowing them to learn from experience and improve over time. **Example in Healthcare:
